In [ ]:
in_s = 'दुहां वइगु ध्यबाया परिमाण अप्वइगु खँ नं अर्थविद्तय्सं धाःगु दु । डलरया भाः थहां वन कि विदेशय् ज्या यानाच्वंपिं नेपाःमितय्सं पेश्की हे तलब कयाः तक नं नेपालय् ध्यबा छ्वा हइगु जूगुलिं आः छुं ई रेमिटेन्स अप्वयेफुगु अनुमान नेपाल राष्ट्र बैंकं याःगु दु ।'
out = in_s.split('। ')[:-1]
print('\n'.join(out))

# 1. clean out gibberish

In [ ]:
# this step shinks 5441 to 5017 lines
text_file = '/Users/bri/dev/NewarOscar/new_dedup.txt'

bad_words = ['Ü','ú', 'ö', 'ó','ő','í','É']

with open(text_file) as oldfile, open('newfile.txt', 'w') as newfile:
    for line in oldfile:
        if not any(bad_word in line for bad_word in bad_words):
            newfile.write(line)

# 2. put each sentence in one line

In [ ]:
TextFile = '/Users/bri/dev/NewarOscar/newfile.txt'

with open(TextFile, 'r') as oldfile, open('lined_up.txt','w') as lined_up:
    for line in oldfile:
        out = line.split('।')
        if out != "":
            lined_up.write('\n'.join(out))

# 3. regex remove empty lines (with Atom) 

In [ ]:
# result in 16694

# 4. remove too short lines 

In [ ]:
# result 16603 lines
TextFile = '/Users/bri/dev/NewarOscar/lined_up.txt'

with open(TextFile, 'r') as oldfile, open('good_length.txt','w') as good_length:
    for line in oldfile:
        if len(line)>3:
            good_length.write(line)

In [ ]:
# delete symbols [...] 16599

## get all ki-sentences

In [ ]:
# ki-sentences 2660 raw
TextFile = '/Users/bri/dev/NewarOscar/good_length.txt'

with open(TextFile, 'r') as oldfile, open('ki.txt','w') as ki_sents:
    for line in oldfile:
        if "कि" in line:
            ki_sents.write(line)
          #  print (len(line))

In [ ]:
# ki-sentences without ,
TextFile = '/Users/bri/dev/NewarOscar/ki.txt'

punc = [',','य्‌']

with open(TextFile, 'r') as oldfile, open('ki_no_punc.txt','w') as ki_no_punc:
    for line in oldfile:
        if not any(elem in line for elem in punc):
            ki_no_punc.write(line)
          #  print (len(line))

## get all dhaka-sentences

In [ ]:
# dhaka-sentences 682 raw
TextFile = '/Users/bri/dev/NewarOscar/good_length.txt'

with open(TextFile, 'r') as oldfile, open('dhaka.txt','w') as dhaka_sents:
    for line in oldfile:
        if "धका" in line:
            dhaka_sents.write(line)
          #  print (len(line))

# 5. Whitespace Tokenizing

In [ ]:
# import WhitespaceTokenizer() method from nltk
from nltk.tokenize import WhitespaceTokenizer
	
# Create a reference variable for Class WhitespaceTokenizer
tk = WhitespaceTokenizer()
	
# tokenize cleaned raw corpus file
with open("good_length.txt") as f, open('good_length_tok.txt','w') as tok:
    content = f.read() # Read the whole file
    lines = content.split('\n') # a list of all sentences
    for line in lines:
        tokens = tk.tokenize(line)
        tok.write(str(tokens))


# 6. formating training data from native speakers' annotation file

In [46]:
# regex space out [] from adjacent words
# regex delete sentence number indexes
# command line to delete original sentence (for example:)
# awk 'NR % 4 != 1' 0_100_annotated-touch.txt > 0_100_dhaka_true_withEn.txt
# awk 'NR % 3 != 0' 0_100_dhaka_true_withEn.txt > 0_100_dhaka_true.txt

In [ ]:
#human annotated format example
sent_1 = '[ फ्रान्सया मेपिं विद्वानतय्सं वय्कःया थ्व सफू व उकिया विषयवस्तुयात न्हूगु विषय धकाः ] ययेकादीगु खँ नं वय्कलं कनादीगु दु'
sent_2 = ['कनादीगु',"UNK"]
    
#labels:

#BEC
#IEC
#OEC
#OEC-V
#IEC-V

## Labeling

In [54]:
import itertools
with open('sp1/0_100_dhaka_true.txt') as f:
    for line1,line2 in itertools.zip_longest(*[f]*2):
        tokens = iter(tk.tokenize(line1))
        current_tag = "OEC"
        for token in tokens:
            if token in line2:    
                # Good: even if the verb is complex (aka spaced out),suffix can still apply to the entire phrase
                # Bad: false positive -V may occur, if one word that has multiple meanings occurring more than one time in the same sentence
                suffix = "-V"
            else:
                suffix = ""
            if token == "[":
                print(next(tokens), "\t"+"BEC"+suffix)
                current_tag = "IEC"
            elif token == "]":
                current_tag = "OEC"
                continue
            else:
                print(token, current_tag+suffix, sep="\t")
        print("")

नाउरुमितयेसं 	BEC-V
थ:गु	IEC
टापुयात	IEC
"नाओएरो"	IEC
धका:	IEC
नां	OEC
बिल,	OEC-V
"नाउरु"	OEC
धा:गु	OEC
खँग्वः	OEC
लिपा	OEC
"नाओएरो"	OEC
खँग्वः	OEC
अपभ्रंश	OEC
जुया	OEC
अंग्रेजी	OEC
परिवर्तन	OEC
जूव:गु	OEC
ख:	OEC

थ्व 	BEC-V
लक्षण	IEC
खनेसाथ	IEC-V
कि	IEC
हे	IEC
थ्व	IEC
हे	IEC
ल्वय्	IEC
जूगु	IEC
धका	IEC
धाये	OEC-V
छिनि	OEC
मखु	OEC

अझ 	BEC-V
वृहद	IEC
कथं	IEC
धायेगु	IEC
खःसा	IEC
थ्व	IEC
विज्ञान	IEC
ब्रह्माण्ड	IEC
गथे	IEC
सनाच्वंगु	IEC-V
दु	IEC
धका	IEC
स्वयेत	OEC-V
याइगु	OEC
प्रकृतिया	OEC
साधारण	OEC
एनालाइसिस	OEC
ख	OEC

2अले 	BEC-V
इमिसं	IEC
मनूतय्‌के	IEC
थथे	IEC
धकाः	IEC
न्‍यन	OEC-V
--	OEC
“यहूदीतय्‌	OEC
जुजु	OEC
जुयाः	OEC
बूम्‍ह	OEC
गन	OEC
दु?	OEC
वय्‌कः	OEC
बूगु	OEC
खँ	OEC
सीकेगु	OEC
छगः	OEC
नगु	OEC
पुर्बय्‌	OEC
खनागुलिं	OEC
वय्‌कःयात	OEC
दर्शन	OEC
यायेत	OEC
जिपिं	OEC
थन	OEC
वयागु	OEC
खः	OEC

4उकिं	OEC
हेरोद	OEC
जुजुं	OEC
तःधंपिं	OEC
पुजाहारीतय्‌त	OEC
व	OEC-V
शास्‍त्रीतय्‌त	OEC
सःताः	OEC-V
थथे	OEC
न्‍यन	OEC
--	OEC
“मुक्ति	OEC
बीम्‍ह	OEC
ख्रीष्‍ट	OEC
गन	OEC
बुइगु?”	OEC
5अले	OEC
इमिसं	OE

रिमसयात	IEC
स्याःगु	IEC-V
धका	IEC
बाखनय	OEC
कनातःगु	OEC-V
दु	OEC-V

थ्व 	BEC-V
पुचःया	IEC
मिजंत	IEC
न्हापा	IEC
तंचाःसां	IEC
लिपा	IEC
थ्व	IEC
थाय्	IEC
च्वनेत	IEC-V
सेल्लाःगु	IEC
थाय्	IEC
धकाः	IEC
सीका	OEC-V
थ्व	OEC
पुचलं	OEC
थन	OEC
हे	OEC
नगर	OEC
दयेकल	OEC

रोमन	OEC
चिनाखँमि	OEC
भर्जिलया	OEC
महाकाव्य	OEC
एनिदय्	OEC
धाःसा	OEC
त्रोयमि 	BEC-V
राजकुमार	IEC
एनियसं	IEC
मिसात	IEC
ताइबर	IEC
खुसिं	IEC
समुद्रय	IEC
लिहां	IEC
मवनिगु	IEC
खं	IEC
न्यंसा	IEC
ताइबर	IEC
खुसिइ	IEC
मच्वंगु	IEC-V
व	IEC-V
लिपा	IEC
एनियसया	IEC
वंशजत	IEC
अल्बान	IEC
(Alban)	IEC
जुजु	IEC
जूगु	IEC
व	IEC-V
थ्व	IEC
हे	IEC
अल्बान	IEC
वंशजं	IEC
रोमुलस	IEC
बुयावःगु	IEC
धका	IEC
च्वयातःगु	OEC-V
दु	OEC-V

छुं 	BEC-V
छगु	IEC
रेजिस्तरय्	IEC
थ्व	IEC
रेजिस्त्यान्स	IEC
भोल्तेज	IEC
स्वया	IEC-V
स्वतन्त्र	IEC
व	IEC-V
मपाइगु	IEC
(constant)	IEC
धका	IEC
नालेगु	OEC-V
या	OEC-V

थ्व 	BEC-V
विजय	IEC
बेल्जियमं	IEC
विश्वकप	IEC
त्याकेफइ	IEC-V
ला	IEC
धकाः	IEC
न्ह्यसः	OEC
तयाच्वनीपिं	OEC
आलोचकतय्	OEC
निंतिं	OEC
नं	OEC
बांलाःगु	OEC
लिसः	OEC
जूगु	OEC-V
दु	OEC

In [ ]:
# import WhitespaceTokenizer() method from nltk
from nltk.tokenize import WhitespaceTokenizer
	
# Create a reference variable for Class WhitespaceTokenizer
tk = WhitespaceTokenizer()

with open ('annotate/formating_output.txt','w') as format_out:
    # Here line up the input sentences
    tokens = tk.tokenize(sent_1)
  #  print(tokens)
    cp_flag = False
    for n in range(0,len(tokens)):
        if tokens[n] == "[":
            cp_flag = True
            if tokens[n+1] == "]":
                cp_flag = False
            else: tokens[n+1] += " " + "BEC" + "\n"
        elif tokens[n] == "]":
                cp_flag = False
        else:
            if cp_flag == True:
                if "BEC" not in tokens[n]:
                    tokens[n] += " " + "IEC" + "\n"
            else: tokens[n] += " " + "OEC" + "\n"
    print(tokens)

## formatting version 2 -no

In [ ]:
# import WhitespaceTokenizer() method from nltk
from nltk.tokenize import WhitespaceTokenizer
	
# Create a reference variable for Class WhitespaceTokenizer
tk = WhitespaceTokenizer()

with open ('annotate/formating_output.txt','w') as format_out:
    # Here line up the input sentences
    tokens = tk.tokenize(sent_1)
  #  print(tokens)
    cp_flag = False
    is_BEC = False
    for n in range(0,len(tokens)):
        if tokens[n] == "[":
            cp_flag = True
        elif tokens[n] == "]":
            cp_flag = False
        else:
            if cp_flag == True and is_BEC == False:
                tokens[n] += " " + "BEC" + "\n"
                is_BEC = True        
            else: 
                if cp_flag == True:
                    tokens[n] += " " + "IEC" + "\n"
                else: tokens[n] += " " + "OEC" + "\n"
    print(tokens)
              

## Version 3 - yes

In [ ]:
sent = "Mary said [ that John went to the store ] , unfortunately ."
verbs = ["said", "went"]

sent_1 = '[ फ्रान्सया मेपिं विद्वानतय्सं वय्कःया थ्व सफू व उकिया विषयवस्तुयात न्हूगु विषय धकाः ] ययेकादीगु खँ नं वय्कलं कनादीगु दु'
verb_1 = ['कनादीगु',"UNK"]

sent_2 = 'थःपिंसं छुं ज्या मयाःसां मेपिंसं धाःसा [ भाषाया लागिं यक्व ज्या यायेमाः धकाः ] आग्रह याइपिं नेवाःत यक्व दुगु नं वय्कःया रिसर्चं सीदुगु दु'
verb_2 = ["सीदुगु दु","यायेमाः"]


# import WhitespaceTokenizer() method from nltk
from nltk.tokenize import WhitespaceTokenizer
	
# Create a reference variable for Class WhitespaceTokenizer
tk = WhitespaceTokenizer()

tokens = iter(tk.tokenize(sent_2))
#tokens = iter(sent_1.split())

current_tag = "OEC"
        
for token in tokens:
    if token in verb_2:
        suffix = "-V"
    else:
        suffix = ""
    if token == "[":
        print(next(tokens), "\t"+"B-EC"+suffix)
        current_tag = "I-EC"
    elif token == "]":
        current_tag = "OEC"
        continue
    else:
        print(token, current_tag+suffix, sep="\t")
        

In [ ]:
import torch
x = torch.rand(5, 3)
print(x)

In [ ]:
pip install inltk

In [ ]:
conda install -c conda-forge spacy

In [ ]:
from inltk.inltk import identify_language

In [ ]:
identify_language("中文")

In [ ]:
from nltk.parse.corenlp import CoreNLPDependencyParser